In [ ]:
# %% Librerías
import numpy as np
import matplotlib.pyplot as plt

# %% Parámetros del algoritmo genético
pob_size = 50      # Tamaño de la población
a = -10            # Valor mínimo de x
b = 10             # Valor máximo de x
decimales = 3      # Número de decimales
n = 2              # Número de variables
epsilon = 1e-6     # Cota de selección
proba_cruza = 0.8  # Probabilidad de cruza
proba_muta = 0.2   # Probabilidad de mutación
generaciones = 50  # Número de generaciones
etapas = 5         # Número de etapas

# %% Función para generar la población en codificación real
def gen_pop_real(pob_size, n, a, b):
    return np.round(np.random.uniform(a, b, (pob_size, n)), decimals=decimales)

# %% Función de evaluación (Himmelblau)
def funcion(x, y):
    return (x**2 + y - 11)**2 + (x + y**2 - 7)**2

# %% Evaluación de la función objetivo
def eva_funcion(ind):
    x, y = ind
    return funcion(x, y)

# %% Selección por torneo
def torneo(pob, epsilon):
    idx1, idx2 = np.random.choice(len(pob), 2, replace=False)
    f1, f2 = eva_funcion(pob[idx1]), eva_funcion(pob[idx2])
    return pob[idx1] if f1 < f2 else pob[idx2]

# %% Cruza aritmética
def cruzar(padre1, padre2, prob_cruza):
    if np.random.rand() < prob_cruza:
        alpha = np.random.rand()  # Coeficiente aleatorio
        hijo1 = alpha * padre1 + (1 - alpha) * padre2
        hijo2 = (1 - alpha) * padre1 + alpha * padre2
        return hijo1, hijo2
    else:
        return padre1.copy(), padre2.copy()

# %% Mutación gaussiana
def mutar(ind, prob_muta, a, b):
    mutante = ind.copy()
    for i in range(len(mutante)):
        if np.random.rand() < prob_muta:
            mutante[i] += np.random.normal(0, 1)  # Perturbación gaussiana
            mutante[i] = np.clip(mutante[i], a, b)  # Mantener dentro del rango
    return np.round(mutante, decimals=decimales)

# %% Algoritmo Genético con codificación real
def AG_real(pob_size, n, a, b, decimales, epsilon, proba_cruza, proba_muta, etapas, generaciones):
    pob = gen_pop_real(pob_size, n, a, b)

    mejor_hijo = None
    mejor_aptitud = np.inf
    h = []      # Historial del mejor
    h_pp = []   # Historial del promedio

    for eta in range(etapas):
        print(f"\n=== Etapa {eta+1} ===")
        for gen in range(generaciones):
            evaluar = np.array([eva_funcion(ind) for ind in pob])
            mejor_ahora = np.min(evaluar)
            h.append(mejor_ahora)
            h_pp.append(np.mean(evaluar))

            if mejor_ahora < mejor_aptitud:
                mejor_aptitud = mejor_ahora
                mejor_hijo = pob[np.argmin(evaluar)]

            if gen % 10 == 0:
                print(f"Generación {gen} - Mejor: {mejor_ahora:.3f} | Promedio: {np.mean(evaluar):.3f}")

            # Selección y reproducción
            nueva_pob = []
            while len(nueva_pob) < pob_size:
                padre1 = torneo(pob, epsilon)
                padre2 = torneo(pob, epsilon)
                hijo1, hijo2 = cruzar(padre1, padre2, proba_cruza)
                hijo1 = mutar(hijo1, proba_muta, a, b)
                hijo2 = mutar(hijo2, proba_muta, a, b)
                nueva_pob.append(hijo1)
                if len(nueva_pob) < pob_size:
                    nueva_pob.append(hijo2)

            pob = np.array(nueva_pob)

    return mejor_hijo, mejor_aptitud, h, h_pp

# %% Ejecutar el algoritmo genético
def main():
    mejor_ind, mejor_apt, h, h_pp = AG_real(
        pob_size, n, a, b, decimales, epsilon, proba_cruza, proba_muta,
        etapas, generaciones)
    
    print("\nResultado final:")
    print("Mejor individuo (x1, x2):", mejor_ind)
    print("Mejor evaluación f(x1, x2):", round(mejor_apt, decimales))

    plt.figure(figsize=(10, 5))
    plt.plot(h, label="Mejor evaluación")
    plt.plot(h_pp, label="Promedio de evaluación")
    plt.xlabel("Generación")
    plt.ylabel("Evaluación")
    plt.title("Evolución del Algoritmo Genético (Codificación Real)")
    plt.legend()
    plt.grid(True)
    plt.show()

if __name__ == "__main__":
    main()
